In [195]:
import os
import torch

import numpy as np
from mustovi_utils import load_gpt_model, gen_sent

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
gpt_model, gpt_tokenizer = load_gpt_model("neo1.3")

In [3]:
import sys
sys.path.append("../StyleCLIP_modular")
from style_clip import Imagine, create_text_path
import argparse

net = "image" # conv, vqgan
args = dict()
args["clip_names"] = ["ViT-B/16", "ViT-B/32"]#, "RN50"]
args["sideX"] = 50
args["sideY"] = 50
args["circular"] = 0
args["model_type"] = "image"

imagine = Imagine(
                save_progress=False,
                open_folder=False,
                save_video=False,
                verbose=False,
                **args
               )

In [4]:
def gpt_create_style(model, tokenizer, theme_words, num_beams=5, prompt_num=5, top_p=0.8):
    if prompt_num == 0:
        prompter = "The name of a matching painter is:"
        #gpt_prompt = f"""I am an artificial intelligence that can tell you a matching artist (even little known ones) for a set of words describing a painting.
        gpt_prompt = f"""The following are lists of words describing art, followed by the name of the artist:

        The art fits these terms: expressionist, beautiful, vibrant. {prompter} Van Gogh.
        The art fits these terms: epic, fantasy, stunning, beautiful, moody. {prompter} Greg Rutkowski.
        The art fits these terms: realistic, beautiful, landscapes, dinosaurs, forgotten civilizations. {prompter} James Gurney.
        The art fits these terms: {theme_words}. {prompter}"""
    if prompt_num == 1:
        prompter = "The name of a matching painter is:"
        gpt_prompt = f"""The following are lists of words describing art, followed by the name of the artist:

        expressionist, beautiful, vibrant. {prompter} Van Gogh.
        epic, fantasy, stunning, beautiful, moody. {prompter} Greg Rutkowski.
        realistic, beautiful, landscapes, dinosaurs, forgotten civilizations. {prompter} James Gurney.
        {theme_words}. {prompter}"""
    elif prompt_num == 2:
        prompter = "Matching visual artist:"
        gpt_prompt = f"""The following are lists of words describing art, followed by the name of the artist:

        expressionist, beautiful, vibrant. {prompter} Van Gogh.
        epic, fantasy, stunning, beautiful, moody. {prompter} Greg Rutkowski.
        realistic, beautiful, landscapes, forgotten civilizations. {prompter} James Gurney.
        {theme_words}. {prompter}"""
    elif prompt_num == 3:
        prompter = "Matching visual artist:"
        gpt_prompt = f"""The following are lists of words describing art, followed by the name of the artist:

        expressionist, beautiful, vibrant. {prompter} Van Gogh.
        epic, fantasy, stunning, moody. {prompter} Greg Rutkowski.
        realistic, beautiful, landscapes, forgotten civilizations. {prompter} James Gurney.
        happy, dreamy, romantic, sensual. {prompter} Gustav Klimt.
        {theme_words}. {prompter}"""
    elif prompt_num == 4:
        prompter = ". Matching artstyle:"
        gpt_prompt = f"""The following are lists of adjectives, listed in order of importance. They are followed by a name of an artstyle that matches them:

        introspective, beautiful, sad{prompter} Impressionism.
        
        expressive, wild, colourful{prompter} Expressionism.
        
        popular, internet{prompter} Trending on artstation.
        
        rendered, detailed, high-quality{prompter} Rendered in unreal engine.
        
        {theme_words}{prompter}"""
    elif prompt_num == 5:
        prompter = ". Matching artstyle:"
        gpt_prompt = f"""The following are adjectives, followed by a matching artstyle:
 realistic, beautiful, landscapes, forgotten civilizations{prompter} By James Gurney.
 introspective, beautiful, sad{prompter} A moody painting.
 expressive, wild, colourful{prompter} An expressionist painting.
 epic, fantasy, stunning, moody{prompter} Illustrated by Greg Rutkowski.
 {theme_words}{prompter}"""
# happy, dreamy, romantic, sensual{prompter} Painted by Gustav Klimt.
# popular, internet{prompter} Trending on artstation.


#gpt_prompt = f"""I am an artificial intelligence that can tell you a matching artist (even little known ones) for a set of words describing a painting.
    #The painting fits these terms: {theme_words}. {prompter}"""
    
    #prompter = "He would suggest the artist:"
    #gpt_prompt = f"""An interview with a world-famous art expert. He can tell you an artist with a matching style for a set of words describing a painting.
    #For painting fiting these terms: {theme_words}. {prompter}"""
    
    #prompter = " This painting has been painted by a painter with the name"
    #gpt_prompt = f"""I would describe this painting as {theme_words}. {prompter}"""

    
    out = model.generate(tokenizer.encode(gpt_prompt, return_tensors="pt").to(model.device),
                         top_p=0.98, do_sample=True, output_scores=True, 
                         return_dict_in_generate=True, 
                         max_length=150,
                         num_beams=num_beams, 
                         no_repeat_ngram_size=2,)
    out_text = tokenizer.decode(out["sequences"].tolist()[0])
    #print(out_text.split("Corresponding image description:")[3:])
    clip_prompt = out_text.split(prompter)[-1].split("\n")[0].split("<|endoftext|>")[0].strip()
    #.split(".")[0].strip().strip("\n")
    #while len(clip_prompt) > 25:
    #    clip_prompt = ".".join(clip_prompt.split(".")[:-1])
    return clip_prompt, out_text#.split(prompter)[-1]

In [793]:
import torch


def gpt_create_prompt(model, tokenizer, merged_top_tags, num_beams=6, top_p=0.80, prompt_num=1):
    if prompt_num == 1:
        prompter = ". Full description:"
        gpt_prompt = f"""The following are adjectives describing a song, followed by a description of the corresponding image:
    fast, sad, dark{prompter} A man running through dark woods while crying.
    loud, techno, electronic, abstract{prompter} Dynamic and vibrant colors forming strong geometric shapes that give the impression of a rave.
    slow, soft, beautiful, sad, quiet{prompter} An old woman who is sitting on a chair with her hands folded in front of her. She is looking at you with a sad expression on her face.
    {merged_top_tags}{prompter}"""
    if prompt_num == 2:
        prompter = ". Full description:"
        gpt_prompt = f"""The following are adjectives describing an image, followed by a full description of the corresponding image:
    fast, sad, dark{prompter} A man is running through dark woods while crying.
    loud, techno, electronic, abstract{prompter} Dynamic and vibrant colors forming strong geometric shapes that give the impression of a rave.
    slow, soft, beautiful, sad, quiet{prompter} An old woman is sitting on a chair in a beautiful garden with her hands folded in front of her. She is looking at you with a sad expression on her face.
    {merged_top_tags}{prompter}"""
    
    out = model.generate(tokenizer.encode(gpt_prompt, return_tensors="pt").to(model.device),
                         top_p=top_p, do_sample=True, output_scores=True, 
                         return_dict_in_generate=True, 
                         max_length=220,
                         num_beams=num_beams, 
                         no_repeat_ngram_size=2,)
    out_text = tokenizer.decode(out["sequences"].tolist()[0])
    #print(out_text.split("Corresponding image description:")[3:])
    clip_prompt = out_text.split(prompter)[-1].split("\n")[0].strip().strip('<|endoftext|>')
    if not clip_prompt.endswith("."):
        clip_prompt = ".".join(clip_prompt.split(".")[:-1]) + "."
    return clip_prompt, out_text

In [794]:
themes = [['chill', 'sexy', 'chillout', 'slow', 'Mellow'],
 ['slow', 'soft', 'sad', 'beautiful', 'quiet'],
 ['beautiful', 'sad', 'slow', 'soft', 'Mellow'],
 ['chillout', 'chill', 'party', 'sexy', 'weird'],
 ['sexy', 'chill', 'slow', 'chillout', 'happy'],
 ['quiet', 'catchy', 'soft', 'happy', 'sad'],
 ['chill', 'slow', 'chillout', 'beautiful', 'Mellow'],
 ['slow', 'soft', 'beautiful', 'sad', 'quiet']]

In [813]:
import importlib
import mustovi_utils
importlib.reload(mustovi_utils)
from mustovi_utils import sample_GPT_with_CLIP
from clip import tokenize

tags = ", ".join(themes[0])
    

In [814]:
img_captioning = False

clip_model = imagine.perceptor.models[0] 
if img_captioning:
    from PIL import Image
    img = "hot-dog.jpg"
    target_clip_feats = clip_model.encode_image(imagine.clip_transform(Image.open(f"../CLIP_playground/base_images/{img}").convert("RGB")).unsqueeze(0).cuda())
    gpt_prompt = ""
else:

    print(tags)
    
    target_clip_feats = clip_model.encode_text(tokenize(tags).to("cuda"))

chill, sexy, chillout, slow, Mellow


In [832]:
from mustovi_utils import load_gpt_model, gen_sent

gpt_model, gpt_tokenizer = load_gpt_model("neo1.3")

prefix = "The following are adjectives describing an image, followed by a full description of the corresponding image:\n "
prompter = ". Full description:"
examples = {"fast, sad, dark": " A man is running through dark woods while crying.",
            "loud, techno, electronic, abstract": " Dynamic and vibrant colors forming strong geometric shapes that resemble a rave.",
            "slow, soft, beautiful, sad, quiet": " An old woman is sitting on a chair in a beautiful garden with her hands folded in front of her. She is looking at you with a sad expression on her face.",
           }
target_text = "depression, excitement"

gen_sent(gpt_model, gpt_tokenizer, clip_model, target_clip_feats, 
         start_text="", p=0.9, 
         prefix=prefix, examples=examples, prompter=prompter, target_text=target_text,
         clip_weight=0.4, 
         clip_temp=0.45, gpt_temp=0.75, out_len=50, v=-1, num_beams=20, return_num=2)

Context:  
 The following are adjectives describing an image, followed by a full description of the corresponding image:
 fast, sad, dark. Full description: A man is running through dark woods while crying.
 loud, techno, electronic, abstract. Full description: Dynamic and vibrant colors forming strong geometric shapes that resemble a rave.
 slow, soft, beautiful, sad, quiet. Full description: An old woman is sitting on a chair in a beautiful garden with her hands folded in front of her. She is looking at you with a sad expression on her face.
 depression, excitement. Full description:


['A man is smiling at the camera and you can see a tear running down his cheek.',
 'A beautiful woman in a beautiful dress is playing with a small child and smiling.']

Context:  
 The following are adjectives, followed by a matching artstyle:
 introspective, beautiful, sad. Matching artstyle: A moody painting.
 expressive, wild, colourful. Matching artstyle: An expressionist painting.
 epic, fantasy, stunning, moody. Matching artstyle: Illustrated by Greg Rutkowski.
 realistic, beautiful, landscapes, forgotten civilizations. Matching artstyle: By James Gurney.
 depression, excitement. Matching artstyle:


['A realistic painting.', 'A painting by Benjamin Buchalter.']

'A woman is sitting on a chair in a chill place with her hands folded in front of her. She is looking at you with a sexy look on her face.'

In [777]:
for sent in df.sort_values("clip_score", ascending=False).iloc[:3]["sent"].tolist():
    print(sent)

 The chill music that fills the air around you.

 A chill out music video featuring sexy chicks with slow, Mellow, slow music.

 A chilled out atmosphere of the album that seems to give the impression of a sex club.



In [778]:
for sent in df.sort_values("gpt_score", ascending=False).iloc[:3]["sent"].tolist():
    print(sent)

 A woman is sitting on a chair in a chill place with her hands folded in front of her. She is looking at you with a sexy look on her face.

 The woman is looking at you with a look of pleasure.

 A beautiful, sad, relaxing, chill out girl looks in your direction with a smile on her face.



In [779]:
for sent in df.sort_values("post_score", ascending=False).iloc[:3]["sent"].tolist():
    print(sent)

 A woman is sitting on a chair in a chill place with her hands folded in front of her. She is looking at you with a sexy look on her face.

 The woman is looking at you with a look of pleasure.

 A woman is in a really chill and sexy mood while sitting on a couch.



In [780]:
print("Final preds according to direction: ")
for sent in df.sort_values("direction_similarity", ascending=False).iloc[:3]["sent"]:
    print(sent)

Final preds according to direction: 
 A woman is sitting on a chair in a chill place with her hands folded in front of her. She is looking at you with a sexy look on her face.

 The picture is in a nice relaxing environment with nature and trees in the background. The colors are still bright and the scene is very relaxing.

 A sexy and calm woman is sitting on a couch and is talking on the phone. The man is standing next to her and he is chill and he is wearing a white shirt and a black tie. The woman is wearing a blue and black dress and


Final pred acc to score: 


' A woman is sitting on a chair in a chill place with her hands folded in front of her. She is looking at you with a sexy look on her face.\n'

In [782]:
tags

'chill, sexy, chillout, slow, Mellow'